# Using Emsembl-REST

## Import statements

In [ ]:
pip install ensembl_rest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26095 sha256=491599db3c6aa02d38040fc88333bc76a55132fdf3075c69943dec4d126a271d
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [ ]:
import ensembl_rest

In [ ]:
help(ensembl_rest)

## Getting data

In [ ]:
import requests
import csv

# Define the gene IDs you are interested in
gene_ids = ["ENSG00000000003", "ENSG00000000005", "ENSG00000000419"]  # Replace with your gene IDs

# Initialize a CSV writer
csv_file = open("ensembl_data_homo_sapiens.csv", "w", newline="")
csv_writer = csv.writer(csv_file)

# Write the header to the CSV file
csv_writer.writerow(["emsembl_gene_id", "transcipt_id", "utr5", "cds", "utr3" ])

# Function to get transcript ID for a given gene ID
def get_sequence(gene_id, seq_type="cdna"):
    return ensembl_rest.sequence_id(id=gene_id, type=seq_type)["seq"]


# Loop through each gene ID and retrieve the data
for gene_id in gene_ids:
    gene_data = ensembl_rest.lookup(
        species='homo sapiens',
        id=gene_id
    )
    # Get transcript ID
    transcript_id = gene_data["canonical_transcript"].split(".")[0]

    # Retrieve CDS sequence
    cds_sequence = get_sequence(transcript_id, "cds")

    # Retrieve 5' UTR and 3' UTR sequences
    # utr5_sequence = get_sequence(transcript_id, "cdna")
    # utr3_sequence = get_sequence(transcript_id, "gene_3utr")
    utr5_sequence = ""
    utr3_sequence = ""

    # Write the row to the CSV file
    csv_writer.writerow([gene_id, transcript_id, utr5_sequence, cds_sequence, utr3_sequence])

# Close the CSV file
csv_file.close()

print("Data extraction complete.")

Data extraction complete.


## GC content

In [ ]:
import pandas as pd

In [ ]:
genetic_data = pd.read_csv("ensembl_data_homo_sapiens.csv")
genetic_data

,emsembl_gene_id,transcipt_id,utr5,cds,utr3
0,ENSG00000000003,ENST00000373020,NaN,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,NaN
1,ENSG00000000005,ENST00000373031,NaN,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,NaN
2,ENSG00000000419,ENST00000371588,NaN,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,NaN


In [ ]:
def compute_gc_content(sequence):

    return (sequence.count("C") + sequence.count("G")) / len(sequence)


genetic_data["cds_gc"] = genetic_data["cds"].apply(compute_gc_content)
genetic_data


,emsembl_gene_id,transcipt_id,utr5,cds,utr3,cds_gc
0,ENSG00000000003,ENST00000373020,NaN,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,NaN,0.401485
1,ENSG00000000005,ENST00000373031,NaN,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,NaN,0.407826
2,ENSG00000000419,ENST00000371588,NaN,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,NaN,0.398301
